In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import nfl_data_py as nfl

# Define the years to import
years = range(2019, 2020)

pbp = nfl.import_pbp_data(years, downcast=True, cache=False, alt_path=None)
weekly = nfl.import_weekly_data(years, downcast=True)
seasonal = nfl.import_seasonal_data(years, s_type='ALL')
ngs = nfl.import_ngs_data(stat_type='passing', years=years)

2019 done.
Downcasting floats.
Downcasting floats.


Attempt to make a dvoa-type metric. Explore how downs impact expected points to make a better epa. 
- Points Above Replacement [PAR]
- First Down Above Replacement [FDAR] how likely are you to obtain another first down on this drive?

Use strength of schedule and matchups to weight player performance. Include situations like 4th quarter.

Can I predict snap count of players? If so, does that help predict points?

Make multiple functions that take in past data (play-by-play) 
- summarize the data by predicting the play outcome (yardage gained, score, 1st down)

Make a function that is able to summarize the effect of each variable on the outcome of the play

So, if each play occurs after the prior; I can make a cumulative sum that tracks the performance of teams
- New features like: Of last 100 plays how many have been touchdowns? How many have been sacks?
- total plays played

Can I feed all columns into a model to predict the future?
- Should I predict the next play? The game result? The next game result? Season win total?
- Can I find the most important features to then summarize for other problems? Can I then reduce the dimensions?
- Or should I just use averages and tendencies to summarize the current data in relation to future data?

It probably makes more sense to build new features first from each play and then by each drive and then by each game, and then by each season. Using the most nuanced and pinpoint data to predict larger and larger outcomes. First build data that describes each player's past tendency and use it to predict the next play. Then use the play outcome over expected to predict the drive, and then use the drive outcome over expected to predict the game and so on.

In [4]:
game_information = [
    'game_id', 'play_id', 'game_date', 'season', 'home_team', 'away_team', 'home_score', 
    'away_score', 'total_home_score', 'total_away_score', 'posteam', 'posteam_type', 'defteam', 
    'location', 'stadium', 'weather', 'temp', 'wind', 'roof', 'surface', 'home_coach', 'away_coach', 
    'game_stadium', 'game_location', 'game_weather', 'game_wind', 'game_temp', 'game_surface', 
    'game_roof', 'game_div_game', 'game_total', 'game_spread_line', 'game_total_line', 'game_result', 
    'game_away_score', 'game_home_score', 'div_game', 'spread_line', 'total_line', 'total', 'result', 
    'explanation']

drive_information = [
    'drive', 'drive_real_start_time', 'drive_play_count', 'drive_time_of_possession', 
    'drive_first_downs', 'drive_inside20', 'drive_ended_with_score', 'drive_quarter_start', 
    'drive_quarter_end', 'drive_yards_penalized', 'drive_start_transition', 'drive_end_transition', 
    'drive_game_clock_start', 'drive_game_clock_end', 'drive_start_yard_line', 'drive_end_yard_line', 
    'drive_play_id_started', 'drive_play_id_ended', 'fixed_drive', 'fixed_drive_result']

playcall_information = ['personnel_off', 'play_type', 'run_location', 'run_gap', 'pass_length', 'pass_location',
                        'personeel_def', 'defenders_in_box']

snap_information = [
    'quarter_seconds_remaining', 'half_seconds_remaining', 'game_seconds_remaining', 'game_half', 
    'quarter_end', 'sp', 'qtr', 'down', 'goal_to_go', 'time', 'yrdln', 'ydstogo', 'ydsnet', 'desc', 
    'play_type', 'yards_gained', 'shotgun', 'no_huddle', 'qb_dropback', 'qb_kneel', 'qb_spike', 
    'qb_scramble', 'pass_length', 'pass_location', 'air_yards', 'yards_after_catch', 'run_location', 
    'run_gap', 'field_goal_result', 'kick_distance', 'extra_point_result', 'two_point_conv_result', 
    'home_timeouts_remaining', 'away_timeouts_remaining', 'timeout', 'timeout_team', 'td_team', 
    'posteam_timeouts_remaining', 'defteam_timeouts_remaining', 'score_differential', 'posteam_score', 
    'defteam_score', 'posteam_score_post', 'defteam_score_post', 'score_differential_post', 
    'no_score_prob', 'opp_fg_prob', 'opp_safety_prob', 'opp_td_prob', 'fg_prob', 'safety_prob', 
    'td_prob', 'extra_point_prob', 'two_point_conversion_prob', 'ep', 'epa', 'total_home_epa', 
    'total_away_epa', 'total_home_rush_epa', 'total_away_rush_epa', 'total_home_pass_epa', 
    'total_away_pass_epa', 'air_epa', 'yac_epa', 'comp_air_epa', 'comp_yac_epa', 'total_home_comp_air_epa', 
    'total_away_comp_air_epa', 'total_home_comp_yac_epa', 'total_away_comp_yac_epa', 'total_home_raw_air_epa', 
    'total_away_raw_air_epa', 'total_home_raw_yac_epa', 'total_away_raw_yac_epa', 'wp', 'def_wp', 
    'home_wp', 'away_wp', 'wpa', 'home_wp_post', 'away_wp_post', 'total_home_rush_wpa', 
    'total_away_rush_wpa', 'total_home_pass_wpa', 'total_away_pass_wpa', 'air_wpa', 'yac_wpa', 
    'comp_air_wpa', 'comp_yac_wpa', 'total_home_comp_air_wpa', 'total_away_comp_air_wpa', 
    'total_home_comp_yac_wpa', 'total_away_comp_yac_wpa', 'total_home_raw_air_wpa', 'total_away_raw_air_wpa', 
    'total_home_raw_yac_wpa', 'total_away_raw_yac_wpa']

player_actions_results = [
    'rush_attempt', 'pass_attempt', 'sack', 'qb_hit', 'qb_hit_1_player_id', 'qb_hit_1_player_name', 
    'qb_hit_2_player_id', 'qb_hit_2_player_name', 'solo_tackle', 'assist_tackle', 'tackled_for_loss', 
    'tackle_for_loss_1_player_id', 'tackle_for_loss_1_player_name', 'tackle_for_loss_2_player_id', 
    'tackle_for_loss_2_player_name', 'forced_fumble_player_1_team', 'forced_fumble_player_1_player_id', 
    'forced_fumble_player_1_player_name', 'forced_fumble_player_2_team', 'forced_fumble_player_2_player_id', 
    'forced_fumble_player_2_player_name', 'fumble_forced', 'fumble_not_forced', 'fumble_out_of_bounds', 
    'fumble_lost', 'fumble_recovery_1_team', 'fumble_recovery_1_yards', 'fumble_recovery_1_player_id', 
    'fumble_recovery_1_player_name', 'fumble_recovery_2_team', 'fumble_recovery_2_yards', 
    'fumble_recovery_2_player_id', 'fumble_recovery_2_player_name', 'interception', 'interception_player_id', 
    'interception_player_name', 'lateral_interception_player_id', 'lateral_interception_player_name', 
    'pass_defense_1_player_id', 'pass_defense_1_player_name', 'pass_defense_2_player_id', 
    'pass_defense_2_player_name', 'punt_blocked', 'punt_inside_twenty', 'punt_in_endzone', 
    'punt_out_of_bounds', 'punt_downed', 'punt_fair_catch', 'kickoff_inside_twenty', 'kickoff_in_endzone', 
    'kickoff_out_of_bounds', 'kickoff_downed', 'kickoff_fair_catch', 'own_kickoff_recovery', 
    'own_kickoff_recovery_td', 'own_kickoff_recovery_player_id', 'own_kickoff_recovery_player_name', 
    'blocked_player_id', 'blocked_player_name', 'punt_returner_player_id', 'punt_returner_player_name', 
    'kickoff_returner_player_id', 'kickoff_returner_player_name', 'punter_player_id', 'punter_player_name', 
    'kicker_player_id', 'kicker_player_name', 'passer_player_id', 'passer_player_name', 'receiver_player_id', 
    'receiver_player_name', 'rusher_player_id', 'rusher_player_name', 'lateral_receiver_player_id', 
    'lateral_receiver_player_name', 'lateral_rusher_player_id', 'lateral_rusher_player_name', 
    'lateral_sack_player_id', 'lateral_sack_player_name', 'assist_tackle_1_player_id', 
    'assist_tackle_1_player_name', 'assist_tackle_1_team', 'assist_tackle_2_player_id', 
    'assist_tackle_2_player_name', 'assist_tackle_2_team', 'assist_tackle_3_player_id', 
    'assist_tackle_3_player_name', 'assist_tackle_3_team', 'assist_tackle_4_player_id', 
    'assist_tackle_4_player_name', 'assist_tackle_4_team', 'solo_tackle_1_team', 'solo_tackle_2_team', 
    'solo_tackle_1_player_id', 'solo_tackle_2_player_id', 'solo_tackle_1_player_name', 
    'solo_tackle_2_player_name', 'fumbled_1_team', 'fumbled_1_player_id', 'fumbled_1_player_name', 
    'fumbled_2_player_id', 'fumbled_2_player_name', 'fumbled_2_team', 'return_team', 'return_yards', 
    'penalty_team', 'penalty_player_id', 'penalty_player_name', 'penalty_yards', 'penalty', 'penalty_type', 
    'replay_or_challenge', 'replay_or_challenge_result', 'defensive_two_point_attempt', 
    'defensive_two_point_conv', 'defensive_extra_point_attempt', 'defensive_extra_point_conv', 
    'first_down_rush', 'first_down_pass', 'first_down_penalty', 'third_down_converted', 'third_down_failed', 
    'fourth_down_converted', 'fourth_down_failed', 'incomplete_pass', 'touchback', 'safety', 
    'tackled_for_loss', 'fumble', 'complete_pass', 'lateral_reception', 'lateral_rush', 'lateral_return', 
    'lateral_recovery', 'touchdown', 'pass_touchdown', 'rush_touchdown', 'return_touchdown', 
    'extra_point_attempt', 'two_point_attempt', 'field_goal_attempt', 'kickoff_attempt', 'punt_attempt']

special_teams = [
    'special_teams_play', 'st_play_type', 'punt_blocked', 'punt_inside_twenty', 'punt_in_endzone', 
    'punt_out_of_bounds', 'punt_downed', 'punt_fair_catch', 'kickoff_inside_twenty', 'kickoff_in_endzone', 
    'kickoff_out_of_bounds', 'kickoff_downed', 'kickoff_fair_catch', 'own_kickoff_recovery', 
    'own_kickoff_recovery_td', 'punt_returner_player_id', 'punt_returner_player_name', 
    'kickoff_returner_player_id', 'kickoff_returner_player_name', 'punter_player_id', 'punter_player_name', 
    'kicker_player_id', 'kicker_player_name', 'blocked_player_id', 'blocked_player_name']

clock_and_time = [
    'start_time', 'time_of_day', 'play_clock', 'end_clock_time', 'drive_game_clock_start', 
    'drive_game_clock_end']

advanced_metrics = [
    'cp', 'cpoe', 'series', 'series_success', 'series_result', 'order_sequence', 'nfl_api_id', 
    'play_deleted', 'play_type_nfl']

In [5]:
# Print the columns
pbp.head()

,play_id,game_id,old_game_id_x,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,offense_players,defense_players,n_offense,n_defense,ngs_air_yards,time_to_throw,was_pressure,route,defense_man_zone_type,defense_coverage_type
0,1.0,2019_01_ATL_MIN,2019090804,MIN,ATL,REG,1,None,None,None,...,,,0.0,0.0,NaN,NaN,NaN,None,None,None
1,36.0,2019_01_ATL_MIN,2019090804,MIN,ATL,REG,1,ATL,away,MIN,...,00-0035150;00-0033336;00-0032918;00-0032422;00...,00-0034411;00-0031570;00-0028042;00-0030968;00...,11.0,11.0,NaN,NaN,NaN,None,None,None
2,51.0,2019_01_ATL_MIN,2019090804,MIN,ATL,REG,1,ATL,away,MIN,...,00-0035235;00-0028042;00-0031279;00-0026997;00...,00-0031256;00-0027885;00-0031546;00-0027701;00...,11.0,11.0,NaN,NaN,NaN,None,None,None
3,79.0,2019_01_ATL_MIN,2019090804,MIN,ATL,REG,1,ATL,away,MIN,...,00-0035235;00-0030968;00-0031279;00-0026997;00...,00-0027885;00-0031256;00-0031546;00-0032389;00...,11.0,11.0,NaN,NaN,NaN,None,None,None
4,100.0,2019_01_ATL_MIN,2019090804,MIN,ATL,REG,1,ATL,away,MIN,...,00-0035235;00-0031279;00-0026997;00-0032392;00...,00-0031256;00-0031546;00-0032389;00-0032448;00...,11.0,11.0,NaN,NaN,NaN,None,None,None


In [6]:
# Print the columns
weekly.columns

Index(['player_id', 'player_name', 'player_display_name', 'position',
       'position_group', 'headshot_url', 'recent_team', 'season', 'week',
       'season_type', 'opponent_team', 'completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr', 'special_teams_tds', 'fantasy_points

In [7]:
# Print the columns
seasonal.columns

Index(['player_id', 'season', 'season_type', 'completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr_x', 'special_teams_tds', 'fantasy_points', 'fantasy_points_ppr',
       'games', 'tgt_sh', 'ay_sh', 'yac_sh', 'wopr_y', 'ry_sh', 'rtd_sh',
       'rfd_sh', 'rtdfd_sh', 'dom', '

In [18]:
ngs.columns

Index(['season', 'season_type', 'week', 'player_display_name',
       'player_position', 'team_abbr', 'avg_time_to_throw',
       'avg_completed_air_yards', 'avg_intended_air_yards',
       'avg_air_yards_differential', 'aggressiveness',
       'max_completed_air_distance', 'avg_air_yards_to_sticks', 'attempts',
       'pass_yards', 'pass_touchdowns', 'interceptions', 'passer_rating',
       'completions', 'completion_percentage',
       'expected_completion_percentage',
       'completion_percentage_above_expectation', 'avg_air_distance',
       'max_air_distance', 'player_gsis_id', 'player_first_name',
       'player_last_name', 'player_jersey_number', 'player_short_name'],
      dtype='object')

In [21]:
pbp[['game_id', 'play_id', 'offense_players']].head()

,game_id,play_id,offense_players
0,2019_01_ATL_MIN,1.0,
1,2019_01_ATL_MIN,36.0,00-0035150;00-0033336;00-0032918;00-0032422;00...
2,2019_01_ATL_MIN,51.0,00-0035235;00-0028042;00-0031279;00-0026997;00...
3,2019_01_ATL_MIN,79.0,00-0035235;00-0030968;00-0031279;00-0026997;00...
4,2019_01_ATL_MIN,100.0,00-0035235;00-0031279;00-0026997;00-0032392;00...
